!!!!!!!!!!!!!!!! DRAFT DRAFT DRAFT !!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!! work in progesss  !!!!!!!!!!!!!!!! 

!!!!!!!!!!!!!!!! DRAFT DRAFT DRAFT !!!!!!!!!!!!!!!!

Credits / Inspired by => https://github.com/mwitiderrick/stockprice

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!pip install mido

In [47]:
import mido
mid = mido.MidiFile('bach.mid')

In [48]:
song = []
full_song_message = []
for i, track in enumerate(mid.tracks):
    if track.name == "Track 1":
        for msg in track:
            print(msg)
            if msg.dict()['type'] in ['note_on','note_off']:
                song.append((msg.dict()['note'],msg.dict()['velocity'],msg.dict()['time'],msg.dict()['type']))
            full_song_message.append(msg)

<meta message track_name name='Track 1' time=0>
program_change channel=0 program=19 time=0
note_on channel=0 note=67 velocity=92 time=0
note_on channel=0 note=72 velocity=92 time=0
note_off channel=0 note=72 velocity=64 time=192
note_on channel=0 note=76 velocity=92 time=0
note_off channel=0 note=76 velocity=64 time=96
note_on channel=0 note=74 velocity=92 time=0
note_off channel=0 note=74 velocity=64 time=96
note_off channel=0 note=67 velocity=64 time=0
note_on channel=0 note=72 velocity=92 time=0
note_off channel=0 note=72 velocity=64 time=192
note_on channel=0 note=67 velocity=92 time=0
note_on channel=0 note=72 velocity=92 time=0
note_on channel=0 note=76 velocity=92 time=0
note_off channel=0 note=76 velocity=64 time=192
note_off channel=0 note=72 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=0
note_on channel=0 note=67 velocity=92 time=0
note_on channel=0 note=74 velocity=92 time=0
note_off channel=0 note=74 velocity=64 time=192
note_on channel=0 note=79 velocity=

In [49]:
song

[(67, 92, 0, 'note_on'),
 (72, 92, 0, 'note_on'),
 (72, 64, 192, 'note_off'),
 (76, 92, 0, 'note_on'),
 (76, 64, 96, 'note_off'),
 (74, 92, 0, 'note_on'),
 (74, 64, 96, 'note_off'),
 (67, 64, 0, 'note_off'),
 (72, 92, 0, 'note_on'),
 (72, 64, 192, 'note_off'),
 (67, 92, 0, 'note_on'),
 (72, 92, 0, 'note_on'),
 (76, 92, 0, 'note_on'),
 (76, 64, 192, 'note_off'),
 (72, 64, 0, 'note_off'),
 (67, 64, 0, 'note_off'),
 (67, 92, 0, 'note_on'),
 (74, 92, 0, 'note_on'),
 (74, 64, 192, 'note_off'),
 (79, 92, 0, 'note_on'),
 (79, 64, 192, 'note_off'),
 (67, 64, 0, 'note_off'),
 (67, 92, 0, 'note_on'),
 (67, 64, 192, 'note_off'),
 (79, 92, 0, 'note_on'),
 (74, 92, 0, 'note_on'),
 (67, 92, 0, 'note_on'),
 (67, 64, 192, 'note_off'),
 (74, 64, 0, 'note_off'),
 (79, 64, 0, 'note_off'),
 (67, 92, 0, 'note_on'),
 (72, 92, 0, 'note_on'),
 (76, 92, 0, 'note_on'),
 (76, 64, 192, 'note_off'),
 (76, 92, 0, 'note_on'),
 (76, 64, 96, 'note_off'),
 (77, 92, 0, 'note_on'),
 (77, 64, 96, 'note_off'),
 (72, 64, 0,

In [50]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)




for note, velocity, time, type in song:
    track.append(mido.Message(type, note=note, velocity=velocity, time=time))
    #track.append(message)
    
mid.save('new_song.mid')

In [56]:
training_set = pd.DataFrame(song)
del training_set[1]
del training_set[2]
del training_set[3]
training_set = training_set.to_numpy()

In [57]:
"""
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
"""
training_set_scaled = training_set
training_set_scaled

array([[67],
       [72],
       [72],
       ...,
       [72],
       [67],
       [64]])

In [59]:
X_train = []
y_train = []
for i in range(60, 2974):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [60]:
X_train

array([[[67],
        [72],
        [72],
        ...,
        [74],
        [74],
        [76]],

       [[72],
        [72],
        [76],
        ...,
        [74],
        [76],
        [76]],

       [[72],
        [76],
        [76],
        ...,
        [76],
        [76],
        [74]],

       ...,

       [[65],
        [64],
        [84],
        ...,
        [64],
        [67],
        [72]],

       [[64],
        [84],
        [84],
        ...,
        [67],
        [72],
        [72]],

       [[84],
        [84],
        [64],
        ...,
        [72],
        [72],
        [67]]])

In [61]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [62]:
model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(LSTM(units = 50))
model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')



Epoch 1/10
2914/2914 [==============================] - 24s 8ms/step - loss: 3971.0271
Epoch 2/10
2914/2914 [==============================] - 23s 8ms/step - loss: 2996.5045
Epoch 3/10
2914/2914 [==============================] - 19s 7ms/step - loss: 2489.1895
Epoch 4/10
2914/2914 [==============================] - 26s 9ms/step - loss: 2069.5571
Epoch 5/10
2914/2914 [==============================] - 19s 6ms/step - loss: 1712.3842
Epoch 6/10
2914/2914 [==============================] - 15s 5ms/step - loss: 1407.6200
Epoch 7/10
2914/2914 [==============================] - 15s 5ms/step - loss: 1148.2875
Epoch 8/10
2914/2914 [==============================] - 14s 5ms/step - loss: 929.5467
Epoch 9/10
2914/2914 [==============================] - 14s 5ms/step - loss: 746.2645
Epoch 10/10
2914/2914 [==============================] - 16s 6ms/step - loss: 594.0202


In [65]:
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
2914/2914 [==============================] - 10s 3ms/step - loss: 468.4469
Epoch 2/100
2914/2914 [==============================] - 14s 5ms/step - loss: 366.9769
Epoch 3/100
2914/2914 [==============================] - 16s 5ms/step - loss: 285.4160
Epoch 4/100
2914/2914 [==============================] - 17s 6ms/step - loss: 220.9638
Epoch 5/100
2914/2914 [==============================] - 15s 5ms/step - loss: 170.4245
Epoch 6/100
2914/2914 [==============================] - 16s 6ms/step - loss: 131.8250
Epoch 7/100
2914/2914 [==============================] - 18s 6ms/step - loss: 102.7235
Epoch 8/100
2914/2914 [==============================] - 16s 5ms/step - loss: 81.2007
Epoch 9/100
2914/2914 [==============================] - 16s 5ms/step - loss: 65.5380
Epoch 10/100
2914/2914 [==============================] - 17s 6ms/step - loss: 54.4310
Epoch 11/100
2914/2914 [==============================] - 15s 5ms/step - loss: 46.7368
Epoch 12/100
2914/2914 [=====================

In [66]:
y_predict = model.predict(X_train)

In [64]:
y_predict

array([[48.93138],
       [48.93138],
       [48.93138],
       ...,
       [48.93138],
       [48.93138],
       [48.93138]], dtype=float32)

In [67]:
created_song = pd.DataFrame(song)

In [68]:
created_song

,0,1,2,3
0,67,92,0,note_on
1,72,92,0,note_on
2,72,64,192,note_off
3,76,92,0,note_on
4,76,64,96,note_off
...,...,...,...,...
2969,67,92,0,note_on
2970,72,92,0,note_on
2971,72,64,384,note_off
2972,67,64,0,note_off
